#### Notebook to create clevr/cogent extracted data

In [1]:
import json
import math
import os
from pprint import pprint

import numpy as np
import pandas as pd
import scipy.stats as statst

In [2]:
# create a function that reads a file in and insert on the first line the following string objects: and save the file again
def insert_string_in_file(file_path, string):
    with open(file_path, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(string.rstrip('\r\n') + '\n' + content)

def insert_comma_after_closing_bracket(file_path):
    with open(file_path, 'r+') as f:
        content = f.read()
        content = content.replace('}', '},')
        f.seek(0, 0)
        f.write(content)

def truncate_last_char(file_path):
    with open(file_path, 'rb+') as f:
        f.seek(-1, os.SEEK_END)
        f.truncate()

def insert_string_in_file2(file_path, string):
    with open(file_path, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(content.rstrip('\r\n') + '\n' + string.rstrip('\r\n') + '\n')

def transform_file_to_json(fpath):
    insert_string_in_file(fpath, '{"objects": [')
    insert_comma_after_closing_bracket(fpath)
    truncate_last_char(fpath)
    insert_string_in_file2(fpath, ']}')


In [ ]:
fpath = os.path.join(os.getenv('HOME'), "Corpora/CLEVR-extracted/scenes/CLEVR_val_000001.json")
transform_file_to_json(fpath)

In [ ]:
base_dir = os.path.join(os.getenv('HOME'), "Corpora/clevr-extracted-colors/scenes/")
input_dir = os.path.join(base_dir, "val")
output_dir = os.path.join(base_dir, "t-val")

for root, dirs, files in os.walk(input_dir):
    for fname in files:
        # if fname is not .DS_Store
        if (not fname.startswith('.')):
            print(fname)
            transform_file_to_json(os.path.join(root, fname))

In [7]:
def find_extrema(scene, extrema):
    for obj in scene['objects']:
        for attr in obj:
            if attr in extrema:
                if obj[attr] < extrema[attr]['min']:
                    extrema[attr]['min'] = obj[attr]
                if obj[attr] > extrema[attr]['max']:
                    extrema[attr]['max'] = obj[attr]
            else:
                extrema[attr] = {'min': obj[attr], 'max': obj[attr]}
    return extrema

In [9]:
base_dir = os.path.join(os.getenv('HOME'), "Corpora/clevr-extracted-colors/scenes/")
input_dir = os.path.join(base_dir, "val")

extrema = {}
for root, dirs, files in os.walk(input_dir):
    for fname in files:
        # if fname is not .DS_Store
        if (not fname.startswith('.')):
            with open(os.path.join(root, fname)) as f:
                scene = json.load(f)
                extrema = find_extrema(scene, extrema)

In [10]:
extrema

{'xpos': {'min': 30.5, 'max': 453.8041076660156},
 'ypos': {'min': 50.053192138671875, 'max': 293.42584228515625},
 'width': {'min': 4.0, 'max': 148.0},
 'height': {'min': 6.0, 'max': 151.99996948242188},
 'angle': {'min': -90.0, 'max': -0.0},
 'corners': {'min': 3, 'max': 8},
 'area': {'min': 47.5, 'max': 17299.0},
 'relative-area': {'min': 0.00030924479166666665, 'max': 0.11262369791666667},
 'bb-area': {'min': 84.0, 'max': 21395.274296201533},
 'bb-area-ratio': {'min': 0.1575647132196773, 'max': 0.9993035396725399},
 'wh-ratio': {'min': 0.175, 'max': 6.866666925743309},
 'color-mean': {'min': [7.225993676682442e-05,
   53.17488314666197,
   38.316723990945704],
  'max': [359.99994774464784, 52.24493731918998, 43.471016822029355]},
 'color-std': {'min': [0.2194593446811935,
   25.090866828390613,
   10.524596715025808],
  'max': [85.39907841674545, 2.654344837939917, 8.820352321222213]},
 'circle-distance': {'min': 0.7737359871882864, 'max': 1.0},
 'white-level': {'min': 0.0, 'max': 

In [12]:
def get_vals(scene, attr):
    vals = []
    for obj in scene['objects']:
        vals.append(obj[attr])
    return vals

In [45]:
base_dir = os.path.join(os.getenv('HOME'), "Corpora/clevr-extracted-colors/scenes/")
input_dir = os.path.join(base_dir, "val")

black = []
white = []
wh_ratio = []
area = []
for root, dirs, files in os.walk(input_dir):
    for fname in files:
        # if fname is not .DS_Store
        if (not fname.startswith('.')):
            with open(os.path.join(root, fname)) as f:
                scene = json.load(f)
                black.extend(get_vals(scene, 'black-level'))
                white.extend(get_vals(scene, 'white-level'))
                wh_ratio.extend(get_vals(scene, 'wh-ratio'))
                area.extend(get_vals(scene, 'area'))

In [47]:
# make the two black and white lists into a pandas dataframe
df = pd.DataFrame({'black': black, 'white': white, 'wh-ratio': wh_ratio, 'area': area})


In [48]:
df.describe()

,black,white,wh-ratio,area
count,97281.000000,97281.000000,97281.000000,97281.000000
mean,0.591259,0.097901,0.957216,3250.479446
std,0.946396,0.447818,0.227302,2709.536134
min,0.000000,0.000000,0.175000,47.500000
25%,0.095603,0.007713,0.818320,1180.500000
50%,0.282149,0.022196,0.966667,2240.500000
75%,0.689185,0.058585,1.025000,4660.500000
max,26.143959,20.289985,6.866667,17299.000000


In [49]:
# get 99th percentile of black column in df
df['area'].quantile(0.95)

8831.0

In [52]:
val = -0.1
# ensure that val is between 0 and 1
val = max(0, min(1, val))
val

0

In [28]:
def normalise_val_in_range(x, a, b):
    return (x - a) / (b - a)

def normalise_val(attr, val):
    match attr:
        case "xpos":
            min1, max1 = 0, 480
        case "ypos":
            min1, max1 = 0, 320
        case "width":
            min1, max1 = 0, 320
        case "height":
            min1, max1 = 0, 480
        case "angle":
            min1, max1 = 0, 90
        case "corners":
            min1, max1 = 3, 8
        case "area":
            min1, max1 = 0, 153600/4
        case "relative-area":
            min1, max1 = 0, 1
        case "bb-area":
            min1, max1 = 0, 153600/4
        case "bb-area-ratio":
            min1, max1 = 0, 1
        case "wh-ratio":
            min1, max1 = 0, 1
        case "circle-distance":
            min1, max1 = 0, 1
        case "white-level":
            min1, max1 = 0, 1
        case "black-level":
            min1, max1 = 0, 1
        case "color-mean-lab-l":
            min1, max1 = 0, 100
        case "color-mean-lab-a":
            min1, max1 = -128, 127
        case "color-mean-lab-b":
            min1, max1 = -128, 127
        case "color-std-lab-l":
            min1, max1 = 0, 100
        case "color-std-lab-a":
            min1, max1 = -128, 127
        case "color-std-lab-b":
            min1, max1 = -128, 127
        case "color-mean-rgb-r":
            min1, max1 = 0, 1
        case "color-mean-rgb-g":
            min1, max1 = 0, 1
        case "color-mean-rgb-b":
            min1, max1 = 0, 1
        case "color-std-rgb-r":
            min1, max1 = 0, 1
        case "color-std-rgb-g":
            min1, max1 = 0, 1
        case "color-std-rgb-b":
            min1, max1 = 0, 1
    
    normalised_val = normalise_val_in_range(val, min1, max1)
    return max(0, min(1, normalised_val))

In [29]:
from skimage import color
def hsv2lab(h, s, v):
    h = h / 360
    s = s / 100
    v = v / 100
    rgb = color.hsv2rgb([h, s, v])
    return color.rgb2lab(rgb)

def hsv2rgb(h, s, v):
    h = h / 360
    s = s / 100
    v = v / 100
    return color.hsv2rgb([h, s, v])

In [32]:
def transform_data(scene, fname):
    new_scene = {
        "image_index": int(fname.split('_')[2].split('.')[0]),
        "image_filename": (fname.rsplit('.', 1)[0] + '.png').lower(),
    }
    new_objects = []
    for object in scene["objects"]:
        new_object ={}
        
        new_object["attributes"] = object
        new_object["description"] = {}

        tmp1 = new_object["attributes"]["color-mean"]
        tmp1 = hsv2lab(tmp1[0], tmp1[1], tmp1[2])
        tmp1rgb = hsv2rgb(tmp1[0], tmp1[1], tmp1[2])

        tmp2 = new_object["attributes"]["color-std"]
        tmp2 = hsv2lab(tmp2[0], tmp2[1], tmp2[2])
        tmp2rgb = hsv2rgb(tmp2[0], tmp2[1], tmp2[2])

        del new_object["attributes"]["color-mean"]
        del new_object["attributes"]["color-std"]
 
        new_object["attributes"]["color-mean-lab-l"] = tmp1[0]
        new_object["attributes"]["color-mean-lab-a"] = tmp1[1]
        new_object["attributes"]["color-mean-lab-b"] = tmp1[2]

        new_object["attributes"]["color-std-lab-l"] = tmp2[0]
        new_object["attributes"]["color-std-lab-a"] = tmp2[1]
        new_object["attributes"]["color-std-lab-b"] = tmp2[2]

        new_object["attributes"]["color-mean-rgb-r"] = tmp1rgb[0]
        new_object["attributes"]["color-mean-rgb-g"] = tmp1rgb[0]
        new_object["attributes"]["color-mean-rgb-b"] = tmp1rgb[0]

        new_object["attributes"]["color-std-rgb-r"] = tmp2rgb[0]
        new_object["attributes"]["color-std-rgb-g"] = tmp2rgb[0]
        new_object["attributes"]["color-std-rgb-b"] = tmp2rgb[0]

        new_object["attributes"]["angle"] = -new_object["attributes"]["angle"]

        for key, val in new_object["attributes"].items():
            norm_val = normalise_val(key, val)
            new_object["attributes"][key] = round(norm_val, 5)
            

        new_objects.append(new_object)
    new_scene["objects"] = new_objects
    return new_scene

In [33]:
base_dir = os.path.join(os.getenv('HOME'), "Corpora/clevr-extracted-colors/scenes/")
input_dir = os.path.join(base_dir, "val")
output_dir = os.path.join(base_dir, "t-val")
os.makedirs(output_dir, exist_ok=True)

for root, dirs, files in os.walk(input_dir):
    for fname in files:
        # if fname is not .DS_Store
        if (not fname.startswith('.')):
            with open(os.path.join(root, fname)) as f:
                scene = json.load(f)
                scene = transform_data(scene, fname)
                
            with open(os.path.join(output_dir, fname.replace("CLEVR", "clevr")), "w") as f:
                json.dump(scene, f)
    